In [1]:
import requests
import pandas as pd
from urllib.parse import urlencode

In [2]:
client_id = "91309bc2b9d2bd5e6187be85fdcc8146"
client_secret = "5f557dd03c8b074ac8b8d76bdb09095148e1d814d6085910a7130c73ddf439cc"
link = "http://localhost:5000/callback"
auth = (client_id, client_secret)
username = "yormama"

# Update dataframe by sorting the MAL completed list by date and adding the ones after 13 Dec

In [ ]:
## insert code to update

# Authorisation  
## Access token lasts one month (last retrieved: 13 Dec)

In [3]:
# Step 1: Generate code verifier and challenge
# You can use a library like 'secrets' to generate a random string for the code verifier
import secrets

print("step 0")
code_verifier = secrets.token_urlsafe(100)[:128]
code_challenge = code_verifier  # Since only the plain method is supported
print("step 1")

# Step 2: Client requests OAuth 2.0 authentication
authorization_url = "https://myanimelist.net/v1/oauth2/authorize"
params = {
    "response_type": "code",
    "client_id": client_id,
    "state": "PUBLISHED",
    "redirect_uri": link,
    "code_challenge": code_challenge,
    "code_challenge_method": "plain",
}

auth_url = f"{authorization_url}?{urlencode(params)}"


print(f"Visit this URL in your browser to authorize the app:\n{auth_url}")

# After the user authorizes the app, MyAnimeList will redirect back to your redirect URI
redirect_uri = input("Enter the redirect URI from the browser: ")

# Parse the authorization code from the redirect URI
authorization_code = input("Enter the authorization code from the redirect URI: ")

# Step 6: Exchange authorization code for refresh and access tokens
token_url = "https://myanimelist.net/v1/oauth2/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

token_payload = {
    "client_id": client_id,
    "grant_type": "authorization_code",
    "code": authorization_code,
    "redirect_uri": link,
    "code_verifier": code_verifier,
}

# Use HTTP Basic Authentication (Scheme 1)
# client_id is used as the username; client_secret is used as the password
token_response = requests.post(token_url, headers=headers, data=token_payload, auth=auth)

print(token_response.json())


step 0
step 1
Visit this URL in your browser to authorize the app:
https://myanimelist.net/v1/oauth2/authorize?response_type=code&client_id=91309bc2b9d2bd5e6187be85fdcc8146&state=PUBLISHED&redirect_uri=http%3A%2F%2Flocalhost%3A5000%2Fcallback&code_challenge=QVcMVWYxE9vTZ2VW6u_vW39hBLUPMJj_VrH6SGRJwnYYxlkj7Gai6SH7lxTzHNZjYXth8zEkHio1v_x-8KV0ZFGbQqHMAkAvLInk5VuoEBHP4yZBLW3nlw5YiaOTY_hF&code_challenge_method=plain
Enter the redirect URI from the browser: http://localhost:5000/callback?code=def50200a898b496d5adb67d41513b9fd45fc3cbb2e60105bd180dc1d808272f1a13ac8fa29ae0ab522b6964671782e57e69ab401446c15e1fa250d0beca46c515db9a7e2fa417720dd541db7969d4410d8186d02788d3dcafd803880fbb07826325922cd15a8bafbbf1e4cbc65f0903aef261963aa065c9c9d2f61994c49349f6ea49ac6e48e9895fbd708b3c03958add44d188413e7aca968b9428bf826879b7944c3068380ff8ffc5c02ccbd5dc21b0bf0e458ea04a0be57ebb5d4f25e44b45984519ca6fc7c84033b3b98242c5f5c4e34eec5da7dc87272741ec95c263085697f4b0bec0dcbba5d2d9c28e248b403d5ac05f4268a7419d504adeb5f6

In [4]:
access_token = token_response.json()['access_token']

# Only need to do the following cells when initialising a new dataset
## Get first page of completed animes list sorted by alphabet

In [5]:
# API endpoint for the anime list
api_url = f"https://api.myanimelist.net/v2/users/{username}/animelist?status=completed"

# Include the access token in the Authorization header
headers = {
    "Authorization": f"Bearer {access_token}",
}

# Make the request to the MyAnimeList API
response = requests.get(api_url, headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    anime_list = response.json()
    print(anime_list)
else:
    print(f"Error: {response.status_code} - {response.text}")


{'data': [{'node': {'id': 52034, 'title': '"Oshi no Ko"', 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/1812/134736.jpg', 'large': 'https://cdn.myanimelist.net/images/anime/1812/134736l.jpg'}}}, {'node': {'id': 38101, 'title': '5-toubun no Hanayome', 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/1819/97947.jpg', 'large': 'https://cdn.myanimelist.net/images/anime/1819/97947l.jpg'}}}, {'node': {'id': 39783, 'title': '5-toubun no Hanayome ∬', 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/1775/109514.jpg', 'large': 'https://cdn.myanimelist.net/images/anime/1775/109514l.jpg'}}}, {'node': {'id': 48548, 'title': '5-toubun no Hanayome Movie', 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/1037/122516.jpg', 'large': 'https://cdn.myanimelist.net/images/anime/1037/122516l.jpg'}}}, {'node': {'id': 49520, 'title': 'Aharen-san wa Hakarenai', 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/

## Transform from json to pandas

In [6]:
anime_list_data = response.json()['data']
    
# Extract relevant information from the JSON dataid
anime_data = [{'title': entry['node']['title'], 'id': entry['node']['id']} for entry in anime_list_data]

# Create a Pandas DataFrame
df = pd.DataFrame(anime_data)

# Display the DataFrame
print(df)

                           title     id
0                   "Oshi no Ko"  52034
1           5-toubun no Hanayome  38101
2         5-toubun no Hanayome ∬  39783
3     5-toubun no Hanayome Movie  48548
4        Aharen-san wa Hakarenai  49520
5                  Ahiru no Sora  37403
6                  AI no Idenshi  53787
7           Amagi Brilliant Park  22147
8             Ansatsu Kyoushitsu  24833
9  Ansatsu Kyoushitsu 2nd Season  30654


## Retrieve the rest of the pages and make a dataframe containing anime title and id

In [7]:
for offset in range(10,1000,10):
    api_url = f"https://api.myanimelist.net/v2/users/{username}/animelist?offset={offset}&status=completed&nsfw=true" 
    # use nsfw=true at the end else some animes will be missed out
    
    # Include the access token in the Authorization header
    headers = {
        "Authorization": f"Bearer {access_token}",
    }

    # Make the request to the MyAnimeList API
    response = requests.get(api_url, headers=headers)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        anime_list = response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        break

    anime_list_data = response.json()['data']

    # Extract relevant information from the JSON dataid
    anime_data = [{'title': entry['node']['title'], 'id': entry['node']['id']} for entry in anime_list_data]

    # Create a Pandas DataFrame
    df1 = pd.DataFrame(anime_data)
    df = pd.concat([df,df1])

In [8]:
df

,title,id
0,"""Oshi no Ko""",52034
1,5-toubun no Hanayome,38101
2,5-toubun no Hanayome ∬,39783
3,5-toubun no Hanayome Movie,48548
4,Aharen-san wa Hakarenai,49520
...,...,...
5,Youkoso Jitsuryoku Shijou Shugi no Kyoushitsu ...,51096
6,Yuri!!! on Ice,32995
7,Yuukoku no Moriarty,40911
8,Yuukoku no Moriarty Part 2,43325


In [9]:
## Animes that were left out because nsfw=true was absent in api_url
## buddy daddies, danmachi ii ova, danmachi iii ova, 
## lycoris, mushoku ii ova, mushoku ii fitz
## prison school, ragnarok ii part 2, uzaki double

In [10]:
df.to_csv("animes_13Dec.csv")

## Get opening and ending songs of each anime by searching their ID

In [20]:
import re

def alpha(input_string):
    # Use a regular expression to match non-alphabetic characters
    pattern = re.compile('[^a-zA-Z\s]')
    
    # Use the pattern to replace non-alphabetic characters with an empty string
    result_string = pattern.sub('', input_string)
    
    return result_string

In [156]:
songs = []

for anime_id in df['id']:
    api_url = f"https://api.myanimelist.net/v2/anime/{anime_id}?fields=title,opening_themes,ending_themes"    

    # Make the request to the MyAnimeList API
    response = requests.get(api_url, headers=headers) #auth = auth is not correct

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        anime_details = response.json()
        ope = anime_details['opening_themes'] if 'opening_themes' in anime_details else []
        end = anime_details['ending_themes'] if 'ending_themes' in anime_details else []
        for i in range(len(ope)):
            boo = alpha(ope[i]['text'])
            songs.append(boo)
        for j in range(len(end)):
            hoo = alpha(end[j]['text'])
            songs.append(hoo)
    else:
        print(f"Error: {response.status_code} - {response.text}")

In [193]:
song_title = []
song_artist = []
for song in songs:
    t = song.split(' by ')
    if len(t[0]) ==0: 
        song_title.append('Non-English')
    else:
        song_title.append(t[0][1:]) if t[0][0] == ' ' else song_title.append(t[0]) 
    
    if len(t) >1:
        song_artist.append(t[1])
    else:
        song_artist.append('NA')

In [194]:
songs_df = pd.DataFrame({'title': song_title, 'artist': song_artist})

In [195]:
a1 = []
for artist in songs_df['artist']:
    a = artist.split(' ')
    a1.append(a[0])
songs_df['nickname'] = a1

In [197]:
songs_df = songs_df.drop('artist', axis = 1)
songs_df = songs_df.drop_duplicates(keep='first')

In [199]:
songs_df.to_csv('songs_13Dec.csv')

### Troubleshooting missing titles

In [153]:
api_url = f"https://api.myanimelist.net/v2/anime/46569?fields=title,opening_themes,ending_themes"    

# Make the request to the MyAnimeList API
response = requests.get(api_url, headers=headers) #auth = auth is not correct

# Check if the request was successful (status code 200)
if response.status_code == 200:
    anime_details = response.json()
    ope = anime_details['opening_themes'] if 'opening_themes' in anime_details else []
    end = anime_details['ending_themes'] if 'ending_themes' in anime_details else []
    for i in range(len(ope)):
        boo = ope[i]['text']
        op.append(boo)
    for j in range(len(end)):
        hoo = alpha(end[j]['text'])
        ed.append(hoo)

In [155]:
op[-1]

'"Ｗ●ＲＫ" by millennium parade and Sheena Ringo'